In [1]:
import pandas as pd
import numpy as np
import re
import pickle
from tqdm.auto import tqdm
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.decomposition import PCA

# Minimal Cleaning


In [2]:
df = pd.read_parquet(r"C:\Users\fashaikh\Desktop\Thesis_main\topic_modeling.parquet")

In [3]:
def clean_tweet(text):
    # Remove mentions
    text = re.sub(r"@user", "", text)

    # Remove URLs
    text = re.sub(r"http", "", text)

    # Remove extra whitespaces
    text = re.sub(r"\s+", " ", text).strip()

    # Remove leading whitespaces
    text = re.sub(r"^\s+", "", text)

    # Replace &amp; with and
    text = re.sub(r"&amp;", "and", text)

    return text

In [4]:
df["clean"] = df["cleanedContent"].apply(clean_tweet)

In [7]:
df = df[["id", "category", "clean"]]

In [8]:
df.to_parquet(r"C:\Users\fashaikh\Desktop\Thesis_main\minimal_cleaned.parquet")

# Precompute Embeddings


In [2]:
df = pd.read_parquet(r"C:\Users\fashaikh\Desktop\Thesis_main\minimal_cleaned.parquet")

In [3]:
text = df["clean"].to_list()

In [4]:
sentence_model = SentenceTransformer("all-MiniLM-L12-v2", device="cuda")

In [5]:
embeddings = sentence_model.encode(text, show_progress_bar=True)

Batches:   0%|          | 0/146440 [00:00<?, ?it/s]

In [6]:
np.save("embeddings.npy", embeddings)

# Rescaling embeddings with PCA


In [2]:
embeddings = np.load(r"C:\Users\fashaikh\Desktop\Thesis_main\embeddings.npy")

In [3]:
def rescale(x, inplace=False):
    """Rescale an embedding so optimization will not have convergence issues."""
    if not inplace:
        x = np.array(x, copy=True)

    x /= np.std(x[:, 0]) * 10000

    return x

In [4]:
# Initialize and rescale PCA embeddings
pca_embeddings = rescale(PCA(n_components=5).fit_transform(embeddings))

In [5]:
np.save("pca_embeddings_5.npy", pca_embeddings)

# Topic Modeling


In [2]:
# pca_embeddings = np.load("pca_embeddings.npy")
embeddings = np.load(r"C:\Users\fashaikh\Desktop\Thesis_main\embeddings.npy")

Base Topic Model


In [7]:
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english", min_df=50)
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
representation_model = KeyBERTInspired()

# Start UMAP from PCA embeddings
umap_model = UMAP(
    n_neighbors=15,
    n_components=5,
    min_dist=0.0,
    metric="cosine",
    unique=True,
    init=pca_embeddings,
    random_state=42,
)

topic_model = BERTopic(
    umap_model=umap_model,
    vectorizer_model=vectorizer_model,
    ctfidf_model=ctfidf_model,
    representation_model=representation_model,
    low_memory=True,
    verbose=True,
    calculate_probabilities=False,
)

In [8]:
topics, _ = topic_model.fit_transform(df["clean"])

Batches:   0%|          | 0/146440 [00:00<?, ?it/s]

2023-07-09 14:03:39,897 - BERTopic - Transformed documents to Embeddings


In [ ]:
topic_model.get_topic_info()

In [ ]:
topic_model.get_topic(0)

In [ ]:
topic_model.get_document_info(text)

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_heatmap()

In [ ]:
topic_model.save(
    "path/to/my/model_dir",
    serialization="safetensors",
    save_ctfidf=True,
    save_embedding_model=sentence_model,
)

# Manual Topic Modeling

In [28]:
df = pd.read_parquet(r"C:\Users\fashaikh\Desktop\Thesis_main\minimal_cleaned.parquet")

In [29]:
category_mapping = {
    0: "Big Cities",
    1: "Exurbs",
    2: "Rural Middle America",
    3: "Evangelical Hubs",
    4: "Graying America",
    5: "Middle Suburbs",
    6: "College Towns",
    7: "Urban Suburbs",
    8: "Hispanic Centers",
    9: "Working Class Country",
    10: "African American South",
    11: "Aging Farmlands",
    12: "Military Posts",
    13: "Native American Lands",
    14: "LDS Enclaves",
}
df['target'] = df['category'].map({v: k for k, v in category_mapping.items()})

In [30]:
docs = df['clean'].to_list()
y = df['target'].to_list()

In [31]:
from bertopic import BERTopic
from bertopic.backend import BaseEmbedder
from bertopic.cluster import BaseCluster
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.dimensionality import BaseDimensionalityReduction

# Prepare our empty sub-models and reduce frequent words while we are at it.
empty_embedding_model = BaseEmbedder()
empty_dimensionality_model = BaseDimensionalityReduction()
empty_cluster_model = BaseCluster()
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english")

# Fit BERTopic without actually performing any clustering
topic_model= BERTopic(
        embedding_model=empty_embedding_model,
        umap_model=empty_dimensionality_model,
        hdbscan_model=empty_cluster_model,
        ctfidf_model=ctfidf_model,
        vectorizer_model=vectorizer_model
)
topics, probs = topic_model.fit_transform(docs, y=y)


2023-07-09 16:50:57,049 - BERTopic - Transformed documents to Embeddings
2023-07-09 16:50:57,050 - BERTopic - The dimensionality reduction algorithm did not contain the `y` parameter and therefore the `y` parameter was not used
2023-07-09 16:50:57,050 - BERTopic - Reduced dimensionality
2023-07-09 16:50:57,989 - BERTopic - Clustered reduced embeddings


In [32]:
# Assign original classes to our topics
df = topic_model.get_topic_info()
df["Class"] = df.Topic.map(category_mapping)
df


,Topic,Count,Name,Representation,Representative_Docs,Class
0,0,3004707,0_blacklivesmatter_blm_black_people,"[blacklivesmatter, blm, black, people, police,...",[hey District reassigned a middle school teach...,Big Cities
1,1,688950,1_blacklivesmatter_people_black_don,"[blacklivesmatter, people, black, don, justice...",[SAY THEIR NAMES! george floyd trayvon martin ...,Exurbs
2,2,275735,2_oh_blm_antifa_ny,"[oh, blm, antifa, ny, like, people, blm antifa...",[Hi could you please sign and share this petit...,Rural Middle America
3,3,140394,3_ok_al_blm_antifa,"[ok, al, blm, antifa, ok blm, ar, sharpton, al...",[Right. You people!?🤦🏽‍♀️ saying you don’t sup...,Evangelical Hubs
4,4,128348,4_ca_blm_id_antifa,"[ca, blm, id, antifa, like, people, just, don,...","[I don't vote R and I support equality, but yo...",Graying America
5,5,104128,5_blm_antifa_like_people,"[blm, antifa, like, people, don, tell, just, w...",[I support the protestors (NOT the rioters) wh...,Middle Suburbs
6,6,81498,6_ny_ar_blm_ms,"[ny, ar, blm, ms, antifa, ar 15, 15, tx, ky, a...",[Another little trump supporter with an AR 15 ...,College Towns
7,7,65176,7_hi_blm_tx_antifa,"[hi, blm, tx, antifa, blacklivesmatter hi, ant...",[Totalitarianism in all forms is Evil! All Fai...,Urban Suburbs
8,8,60391,8_ar_ar 15_15_blm,"[ar, ar 15, 15, blm, antifa, tx, antifa blm, w...",[Best if viewed on Desktop or to your home the...,Hispanic Centers
9,9,54507,9_oh_ar_ar 15_15,"[oh, ar, ar 15, 15, blm oh, blm, tn, oh right,...","[ATTN: suburban and urban women, children, and...",Working Class Country


# Topic Modeling per class

In [178]:
df = pd.read_parquet(r"C:\Users\fashaikh\Desktop\Thesis_main\minimal_cleaned.parquet")
df = df[df['category'] == 'Hispanic Centers']
df = df.drop_duplicates(subset=['clean'])

In [179]:
docs = df['clean'].to_list()

In [180]:
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english")
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
representation_model = KeyBERTInspired()

umap_model = UMAP(n_neighbors=15, n_components=10, metric='cosine', random_state=42)

hdbscan_model = HDBSCAN(min_cluster_size=30, metric='euclidean', prediction_data=True)

topic_model = BERTopic(nr_topics='auto',
                        top_n_words=5,
                        umap_model=umap_model,
                        hdbscan_model=hdbscan_model,
                        vectorizer_model=vectorizer_model,
                        ctfidf_model=ctfidf_model,
                        representation_model=representation_model,
                        verbose=True,
)

In [188]:
topics, probs = topic_model.fit_transform(docs)

Batches:   0%|          | 0/1813 [00:00<?, ?it/s]

2023-08-13 01:11:05,768 - BERTopic - Transformed documents to Embeddings
2023-08-13 01:12:16,919 - BERTopic - Reduced dimensionality
2023-08-13 01:12:26,641 - BERTopic - Clustered reduced embeddings
2023-08-13 01:12:51,552 - BERTopic - Reduced number of topics from 106 to 70


In [190]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,29985,-1_blm protests_blm movement_protest_riots,"[blm protests, blm movement, protest, riots, p...",[BIDEN shouldn't just be voted in simply bcuz ...
1,0,14296,0_blm antifa_antifa blm_antifa_antifablm,"[blm antifa, antifa blm, antifa, antifablm, de...","[Joe Biden, please condemn BLM and Antifa . I’..."
2,1,3248,1_blm protesters_blm protest_blm protestors_pr...,"[blm protesters, blm protest, blm protestors, ...",[Via : ‘Sending a message to violent MAGAs’: T...
3,2,1062,2_shes blm_blm shes_blm girl_shes racist,"[shes blm, blm shes, blm girl, shes racist, su...",[She rb that at 14 she is 23 and have been sup...
4,3,717,3_white supremacists_antifa proud_proudboys_bl...,"[white supremacists, antifa proud, proudboys, ...",[White supremacy is AWFUL (although Joe Biden ...
...,...,...,...,...,...
65,64,33,64_hk protesters_hk protests_kong protesters_k...,"[hk protesters, hk protests, kong protesters, ...","[This current crop know nothing but MSM, Chine..."
66,65,32,65_jacob blake_jacob blakes_reports blake_blak...,"[jacob blake, jacob blakes, reports blake, bla...",[The #NYPost reports #Blake was violating a re...
67,66,31,66_rice killed_ar15 murdered_rice shot_ar15 ki...,"[rice killed, ar15 murdered, rice shot, ar15 k...",[Tamir Rice was playing with a toy gun and was...
68,67,31,67_ar art_ar exhibition_virtual art_vr ar,"[ar art, ar exhibition, virtual art, vr ar, ar...",[The Top 10 Cultural Heritage and History #3D ...


In [64]:
topic_model.get_topic(2)

[('oppression black', 0.50330204),
 ('black assaulted', 0.5017138),
 ('lie blacks', 0.47917986),
 ('blacks hunted', 0.47729108),
 ('systemic oppression', 0.41862455),
 ('black players', 0.40518874),
 ('vs blm', 0.3842528),
 ('assaulted cops', 0.37712428),
 ('cop shooting', 0.3712714),
 ('support blm', 0.36480564)]

In [193]:
df_topic = topic_model.get_document_info(docs)

In [186]:
topics_to_merge = [41,67]
topic_model.merge_topics(docs, topics_to_merge)

In [209]:
topic_model.set_topic_labels({1:'Protests', 22:'Defund Police', 51:'Education', 6:'Political Persuasion', 23:'Education', 16:'NFL Solidarity', 19:'George Floyd Death', 9:'BLM Covid Spike'})

In [210]:
topic_model.visualize_barchart(topics=[51, 19, 6, 8, 16, 22, 1, 9], title="Topics - Hispanic Centers", width=450, custom_labels=True)

In [127]:
topic_model.visualize_topics()

In [11]:
hierarchical_topics = topic_model.hierarchical_topics(docs)


100%|██████████| 44/44 [00:22<00:00,  1.92it/s]


In [12]:
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)


In [94]:
tree = topic_model.get_topic_tree(hierarchical_topics)

In [95]:
print(tree)

.
├─racial gerrymandering_racial gerrymandering dont_racial gerrymandering dont deserve_voter suppressio
│    ├─■──suppression andor racial gerrymandering_states voter suppression_states voter suppression andor_raci ── Topic: 50
│    └─■──voter suppression states_voter suppression states dont_dont deserve athletes color_states dont deser ── Topic: 26
└─blm movement_blm ut_ut blm_blm_support blm
     ├─ut athletics_ut athletes_ut austin students_black students_black jerseys
     │    ├─black students_black student_racial justice_racism_blacklivesmatter ut
     │    │    ├─racistsexist government blm_black student_racistsexist government_black students_diversity community
     │    │    │    ├─■──raise awareness race racism_recently shared thoughts blacklivesmatter_received streams donated black ── Topic: 18
     │    │    │    └─■──project blm doing social_project blm doing_project blm_protests university supported black_black stu ── Topic: 20
     │    │    └─racial justice_austin ackn